# RTW MAM Model V1 
# County Data First

In [171]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import csv
import pickle

In [172]:
#SET DATE FILTERS & FileName

file = 'CountyMAM2_71.csv'
state_file = 'State_MAM_630_NEWSCORING_V3.csv'

preval_start = '2020-06-17'
preval_end = '2020-06-30'

curr_start = '2020-06-24'
curr_end = '2020-06-30'

prev_start = '2020-06-17'
prev_end = '2020-06-24'

curr_test_start = [int(20200630)]
curr_test_end = [int(20200624)]
prev_test_start = [int(20200623)]
prev_test_end = [int(20200617)]

In [173]:
#UPDATES NEEDED - bring in base file, within base file add in state 2 digit codes
#Need to add in the NYC FIPS dummy code

#Bring in County Data set
nytimes = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
counties = pd.read_csv(nytimes,dtype={'fips': str})
print(counties.head())



base = "https://raw.githubusercontent.com/cbmeco/COVID_MAM/master/base_rtw_v1.csv"
pop_df = pd.read_csv(base,
                dtype={'fips': str})

state_base = "https://raw.githubusercontent.com/cbmeco/COVID_MAM/master/state_base.csv"
state_pop_df = pd.read_csv(state_base)
print(state_pop_df.head())

#pop_df = pd.read_csv("base_rtw_v1.csv",
#                dtype={'fips': str})

#base['fips2'] = np.where(base['county']=='New York City','36067',base['fips'])

counties.loc[counties['county'] == 'New York City', 'fips'] = '36067'


print(counties.head())

         date     county       state   fips  cases  deaths
0  2020-01-21  Snohomish  Washington  53061      1       0
1  2020-01-22  Snohomish  Washington  53061      1       0
2  2020-01-23  Snohomish  Washington  53061      1       0
3  2020-01-24       Cook    Illinois  17031      1       0
4  2020-01-24  Snohomish  Washington  53061      1       0
        State state_code  Comcast Employees  state_pop  CAE  Cable Stores  \
0     Alabama         AL                407    4908621   65            40   
1      Alaska         AK                  0    3038999    0             0   
2     Arizona         AZ                744    7378494  237            17   
3    Arkansas         AR                117    3038999    1            13   
4  California         CA               5022   39937489    0           491   

   Business Services  Tech Ops  comcast_state  
0                  9       178              1  
1                  0         0              0  
2                  5        87         

# Create Active Case Count

In [174]:
#Create new cases field
#Sort df by Fips and Date
counties.sort_values(by=['fips','date'],inplace=True, ascending=True)
print(counties.head())

#Take difference between rows for new case and new death numbers
counties['case_diff'] = counties.cases.diff()
counties['death_diff'] = counties.deaths.diff()


print(counties.head())

             date   county    state   fips  cases  deaths
9480   2020-03-24  Autauga  Alabama  01001      1       0
10835  2020-03-25  Autauga  Alabama  01001      4       0
12367  2020-03-26  Autauga  Alabama  01001      6       0
14025  2020-03-27  Autauga  Alabama  01001      6       0
15803  2020-03-28  Autauga  Alabama  01001      6       0
             date   county    state   fips  cases  deaths  case_diff  \
9480   2020-03-24  Autauga  Alabama  01001      1       0        NaN   
10835  2020-03-25  Autauga  Alabama  01001      4       0        3.0   
12367  2020-03-26  Autauga  Alabama  01001      6       0        2.0   
14025  2020-03-27  Autauga  Alabama  01001      6       0        0.0   
15803  2020-03-28  Autauga  Alabama  01001      6       0        0.0   

       death_diff  
9480          NaN  
10835         0.0  
12367         0.0  
14025         0.0  
15803         0.0  


In [175]:
#Filter for past 14 days
#range_14 = counties[counties['date']>=preval_start]

range_14 = counties[(counties['date']>=preval_start) & (counties['date']<=preval_end)]

print(range_14.head(15))

              date   county    state   fips  cases  deaths  case_diff  \
242845  2020-06-17  Autauga  Alabama  01001    375       7        2.0   
245877  2020-06-18  Autauga  Alabama  01001    400       8       25.0   
248912  2020-06-19  Autauga  Alabama  01001    411       8       11.0   
251951  2020-06-20  Autauga  Alabama  01001    431       9       20.0   
254998  2020-06-21  Autauga  Alabama  01001    434       9        3.0   
258045  2020-06-22  Autauga  Alabama  01001    442       9        8.0   
261094  2020-06-23  Autauga  Alabama  01001    453       9       11.0   
264145  2020-06-24  Autauga  Alabama  01001    469      11       16.0   
267200  2020-06-25  Autauga  Alabama  01001    479      11       10.0   
270261  2020-06-26  Autauga  Alabama  01001    488      11        9.0   
273330  2020-06-27  Autauga  Alabama  01001    498      12       10.0   
276399  2020-06-28  Autauga  Alabama  01001    503      12        5.0   
279467  2020-06-29  Autauga  Alabama  01001    527 

In [176]:
#Group by FIPS, sum up new Cases for prevalence - for 7 day averages will take mean after filtering for 7 days
prevalence_grouped = range_14.groupby(['fips'], as_index=False).sum()

print(prevalence_grouped.head(25))

     fips  cases  deaths  case_diff  death_diff
0   01001   6447     140      164.0         5.0
1   01003   6852     130      291.0         1.0
2   01005   4067      14       80.0         0.0
3   01007   1983      14       54.0         0.0
4   01009   2308      14       87.0         0.0
5   01011   4749     143       55.0         1.0
6   01013   8176     370       56.0         2.0
7   01015   3207      64       70.0         1.0
8   01017   7541     377      105.0         1.0
9   01019    874      90       21.0         2.0
10  01021   2529      41       70.0         1.0
11  01023   2619     165       14.0         1.0
12  01025   3248      63       70.0         2.0
13  01027    495      28       23.0         0.0
14  01029    334      14        7.0         0.0
15  01031   4580      15       57.0         1.0
16  01033   4534      70       96.0         0.0
17  01035   2091      32       71.0         5.0
18  01037    722      14        9.0         0.0
19  01039   3579      45      131.0     

In [177]:
#Join in Prevalence to 
prevalence_w_pop = pd.merge(pop_df,
                 prevalence_grouped[['fips','case_diff']],
                 on='fips', 
                how='left')
print(prevalence_w_pop.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  Total CCC  \
0                0.0    0.0           0.0                0.0        0.0   
1                0.0    0.0           0.0                2.0       98.0   
2              965.0  356.

In [178]:
prevalence_w_pop['active_cases_100k'] = ((prevalence_w_pop['case_diff']/prevalence_w_pop['pop'])*100000)
print(prevalence_w_pop.head())

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  Total CCC  \
0                0.0    0.0           0.0                0.0        0.0   
1                0.0    0.0           0.0                2.0       98.0   
2              965.0  356.0         201.0              146.0     3640.0   
3                0.0    0.0           0.0                0.0        0.0   
4                0.0    0.0           0.0                3.0        9.0   

        Geo Area %Retail Closed in County Updated 5/26      Division  \
0            NaN                                   NaN     Northeast

# Create 7 Day Rolling Avgs for Cases & Deaths

In [179]:
#Filter Diff datasets for current and previous 7 day periods

#join prev and current together with pop

range_current7 = counties[(counties['date']>=curr_start) & (counties['date']<=curr_end)]

range_prev7 = counties[(counties['date']>=prev_start) & (counties['date']<=prev_end)]

print(range_prev7.head(10))

              date   county    state   fips  cases  deaths  case_diff  \
242845  2020-06-17  Autauga  Alabama  01001    375       7        2.0   
245877  2020-06-18  Autauga  Alabama  01001    400       8       25.0   
248912  2020-06-19  Autauga  Alabama  01001    411       8       11.0   
251951  2020-06-20  Autauga  Alabama  01001    431       9       20.0   
254998  2020-06-21  Autauga  Alabama  01001    434       9        3.0   
258045  2020-06-22  Autauga  Alabama  01001    442       9        8.0   
261094  2020-06-23  Autauga  Alabama  01001    453       9       11.0   
264145  2020-06-24  Autauga  Alabama  01001    469      11       16.0   
242846  2020-06-17  Baldwin  Alabama  01003    392       9        3.0   
245878  2020-06-18  Baldwin  Alabama  01003    401       9        9.0   

        death_diff  
242845         0.0  
245877         1.0  
248912         0.0  
251951         1.0  
254998         0.0  
258045         0.0  
261094         0.0  
264145         2.0  
242846 

In [180]:
#Group by with mean
current_grouped = range_current7.groupby(['fips'],as_index=False).mean()
previous_grouped = range_prev7.groupby(['fips'], as_index=False).mean()

current_grouped['curr7_case'] = current_grouped['case_diff']
current_grouped['curr7_death'] = current_grouped['death_diff']
previous_grouped['prev7_case'] = previous_grouped['case_diff']
previous_grouped['prev7_death'] = previous_grouped['death_diff']
print(current_grouped.head())
print(previous_grouped.head())

    fips       cases     deaths  case_diff  death_diff  curr7_case  \
0  01001  500.142857  11.571429  12.000000    0.428571   12.000000   
1  01003  558.428571   9.571429  32.857143    0.142857   32.857143   
2  01005  312.285714   1.000000   6.428571    0.000000    6.428571   
3  01007  157.285714   1.000000   5.000000    0.000000    5.000000   
4  01009  187.428571   1.000000   7.000000    0.000000    7.000000   

   curr7_death  
0     0.428571  
1     0.142857  
2     0.000000  
3     0.000000  
4     0.000000  
    fips    cases  deaths  case_diff  death_diff  prev7_case  prev7_death
0  01001  426.875    8.75     12.000         0.5      12.000          0.5
1  01003  425.875    9.00      9.375         0.0       9.375          0.0
2  01005  271.125    1.00      5.375         0.0       5.375          0.0
3  01007  127.875    1.00      3.125         0.0       3.125          0.0
4  01009  145.500    1.00      5.875         0.0       5.875          0.0


In [181]:
#Merge both sets with population
current_w_pop = pd.merge(pop_df,
                 current_grouped[['fips','curr7_case', 'curr7_death']],
                 on='fips', 
                how='left')
print(current_w_pop.head(10))

previous_w_pop = pd.merge(pop_df,
                 previous_grouped[['fips','prev7_case', 'prev7_death']],
                 on='fips', 
                how='left')
print(previous_w_pop.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  Total CCC  \
0                0.0    0.0           0.0                0.0        0.0   
1                0.0    0.0           0.0                2.0       98.0   
2              965.0  356.

In [182]:
current_w_pop['cur_7rollavg_cases'] = ((current_w_pop['curr7_case']/current_w_pop['pop'])*100000)
current_w_pop['cur_7rollavg_deaths'] = ((current_w_pop['curr7_death']/current_w_pop['pop'])*100000)

In [183]:
previous_w_pop['prev_7rollavg_cases'] = ((previous_w_pop['prev7_case']/previous_w_pop['pop'])*100000)
previous_w_pop['prev_7rollavg_deaths'] = ((previous_w_pop['prev7_death']/previous_w_pop['pop'])*100000)

In [184]:
#Merge with prevalence dataset
prev_curr = pd.merge(prevalence_w_pop,
                 current_w_pop[['fips','cur_7rollavg_cases', 'cur_7rollavg_deaths','curr7_case','curr7_death']],
                 on='fips', 
                how='left')
print(prev_curr.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services     ...       \
0                0.0    0.0           0.0                0.0     ...        
1                0.0    0.0           0.0                2.0     ...        
2              965.0

In [185]:
county_set = pd.merge(prev_curr,
                 previous_w_pop[['fips','prev_7rollavg_cases', 'prev_7rollavg_deaths','prev7_case','prev7_death']],
                 on='fips', 
                how='left')
print(county_set.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services     ...       \
0                0.0    0.0           0.0                0.0     ...        
1                0.0    0.0           0.0                2.0     ...        
2              965.0

In [186]:
#Create indicators for rolling averages
county_set['roll7case_diff'] = ((county_set['cur_7rollavg_cases']-county_set['prev_7rollavg_cases'])/county_set['prev_7rollavg_cases'])
county_set['roll7death_diff'] = ((county_set['cur_7rollavg_deaths']-county_set['prev_7rollavg_deaths'])/county_set['prev_7rollavg_deaths'])

print(county_set.head())

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services       ...         \
0                0.0    0.0           0.0                0.0       ...          
1                0.0    0.0           0.0                2.0       ...          
2              965.0  356.0         201.0              146.0       ...          
3                0.0    0.0           0.0                0.0       ...          
4                0.0    0.0           0.0                3.0       ...          

   cur_7rollavg_cases cur_7rollavg_deaths   curr7_case curr7_death  \
0                 NaN             

# State Level Data

In [187]:
# Pull in NYTimes state data, diff, do same 7 day rolling avg counts
# pull in COVID tracking test data. calculate pos % then rolling avgs
# Manually pull in Rt, might drop this metric

In [188]:
#Bring in state level NYT Data
nytimes_state = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv"
state= pd.read_csv(nytimes_state)
print(state.head())

         date       state  fips  cases  deaths
0  2020-01-21  Washington    53      1       0
1  2020-01-22  Washington    53      1       0
2  2020-01-23  Washington    53      1       0
3  2020-01-24    Illinois    17      1       0
4  2020-01-24  Washington    53      1       0


In [189]:
#Create new cases field
#Sort df by Fips and Date
state['State'] = state['state']
state.sort_values(by=['fips','date'],inplace=True, ascending=True)
print(state.head())

#Take difference between rows for new case and new death numbers
state['case_diff'] = state.cases.diff()
state['death_diff'] = state.deaths.diff()
print(state.head())

           date    state  fips  cases  deaths    State
586  2020-03-13  Alabama     1      6       0  Alabama
637  2020-03-14  Alabama     1     12       0  Alabama
689  2020-03-15  Alabama     1     23       0  Alabama
742  2020-03-16  Alabama     1     29       0  Alabama
795  2020-03-17  Alabama     1     39       0  Alabama
           date    state  fips  cases  deaths    State  case_diff  death_diff
586  2020-03-13  Alabama     1      6       0  Alabama        NaN         NaN
637  2020-03-14  Alabama     1     12       0  Alabama        6.0         0.0
689  2020-03-15  Alabama     1     23       0  Alabama       11.0         0.0
742  2020-03-16  Alabama     1     29       0  Alabama        6.0         0.0
795  2020-03-17  Alabama     1     39       0  Alabama       10.0         0.0


In [190]:
##CREATE State Prevalence measure
#Filter for past 14 days

state_range_14 = state[(state['date']>=preval_start) & (state['date']<=preval_end)]

print(state_range_14.head())

            date    state  fips  cases  deaths    State  case_diff  death_diff
5844  2020-06-17  Alabama     1  27312     790  Alabama      400.0         5.0
5899  2020-06-18  Alabama     1  28206     810  Alabama      894.0        20.0
5954  2020-06-19  Alabama     1  29002     822  Alabama      796.0        12.0
6009  2020-06-20  Alabama     1  29549     838  Alabama      547.0        16.0
6064  2020-06-21  Alabama     1  30021     839  Alabama      472.0         1.0


In [191]:
#create state group for prevalence

state_prevalence_grouped = state_range_14.groupby(['State'], as_index=False).sum()

state_prevalence_grouped['state_active'] = state_prevalence_grouped['case_diff']
print(state_prevalence_grouped.head())

        State  fips    cases  deaths  case_diff  death_diff  state_active
0     Alabama    14   450838   12215    11133.0       165.0       11133.0
1      Alaska    28    13072     150      381.0         2.0         381.0
2     Arizona    56   836126   20227    40101.0       417.0       40101.0
3    Arkansas    70   239968    3319     7586.0        82.0        7586.0
4  California    84  2719564   79588    73022.0       881.0       73022.0


In [192]:
#Take Current and previous rolling averages

state_range_current7 = state[(state['date']>=curr_start) & (state['date']<=curr_end)]
state_range_prev7 = state[(state['date']>=prev_start) & (state['date']<=prev_end)]

print(state_range_current7.head(8))

            date    state  fips  cases  deaths    State  case_diff  death_diff
6229  2020-06-24  Alabama     1  32064     891  Alabama      967.0        27.0
6284  2020-06-25  Alabama     1  33206     896  Alabama     1142.0         5.0
6339  2020-06-26  Alabama     1  34183     907  Alabama      977.0        11.0
6394  2020-06-27  Alabama     1  35083     919  Alabama      900.0        12.0
6449  2020-06-28  Alabama     1  35441     919  Alabama      358.0         0.0
6504  2020-06-29  Alabama     1  37175     929  Alabama     1734.0        10.0
6559  2020-06-30  Alabama     1  38045     950  Alabama      870.0        21.0
6230  2020-06-24   Alaska     2    926      10   Alaska       21.0         0.0


In [193]:
#Groupby Means
#Group by with mean
state_current_grouped = state_range_current7.groupby(['State'],as_index=False).mean()
state_previous_grouped = state_range_prev7.groupby(['State'], as_index=False).mean()

state_current_grouped['state_curr7_case'] = state_current_grouped['case_diff']
state_previous_grouped['state_prev7_case'] = state_previous_grouped['case_diff']
state_current_grouped['state_curr7_death'] = state_current_grouped['death_diff']
state_previous_grouped['state_prev7_death'] = state_previous_grouped['death_diff']

print(state_current_grouped.head())
print(state_previous_grouped.head())

        State  fips          cases       deaths    case_diff  death_diff  \
0     Alabama   1.0   35028.142857   915.857143   992.571429   12.285714   
1      Alaska   2.0    1028.857143    11.428571    32.428571    0.285714   
2     Arizona   4.0   69756.285714  1559.571429  3000.285714   35.285714   
3    Arkansas   5.0   19191.285714   255.285714   585.571429    4.714286   
4  California   6.0  212697.857143  5901.571429  5873.428571   63.714286   

   state_curr7_case  state_curr7_death  
0        992.571429          12.285714  
1         32.428571           0.285714  
2       3000.285714          35.285714  
3        585.571429           4.714286  
4       5873.428571          63.714286  
        State  fips       cases    deaths  case_diff  death_diff  \
0     Alabama   1.0   29713.125   836.875    644.000      13.250   
1      Alaska   2.0     849.500    10.000     21.875       0.000   
2     Arizona   4.0   51004.875  1347.125   2613.625      29.875   
3    Arkansas   5.0   153

In [194]:
#Merge both sets with base file

state_active= pd.merge(county_set,
                 state_prevalence_grouped[['State','state_active',
                                       ]],
                 on='State', 
                how='left')
#print(state_current_w_pop.head(10))

state_current_w_pop = pd.merge(state_active,
                 state_current_grouped[['State','state_curr7_case',
                                       'state_curr7_death']],
                 on='State', 
                how='left')
print(state_current_w_pop.head(10))

county_state_df = pd.merge(state_current_w_pop,
                 state_previous_grouped[['State','state_prev7_case','state_prev7_death']],
                 on='State', 
                how='left')
#print(state_previous_w_pop.head(10))

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   
5  06073      San Diego  California         CA     NaN         Y   
6  06059         Orange  California         CA     NaN       NaN   
7  12086     Miami-Dade     Florida         FL     NaN         Y   
8  48107         Crosby       Texas         TX     NaN       NaN   
9  06065      Riverside  California         CA     NaN       NaN   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  \
0                0.0    0.0           0.0                0.0   
1                0.0    0.0           0.0                2.0   
2              965.0  356.0         201.0              146.

In [195]:
#Create state rolling avg indicator
#####NEED TO ADD IN STATE POP to create per 100k 

county_state_df['state_curr7_case100k'] = ((county_state_df['state_curr7_case']/county_state_df['state_pop'])*100000)
county_state_df['state_curr7_death100k'] = ((county_state_df['state_curr7_death']/county_state_df['state_pop'])*100000)
county_state_df['state_prev7_case100k'] = ((county_state_df['state_prev7_case']/county_state_df['state_pop'])*100000)
county_state_df['state_prev7_death100k'] = ((county_state_df['state_prev7_death']/county_state_df['state_pop'])*100000)


county_state_df['state_roll7case_diff'] = ((county_state_df['state_curr7_case100k']-county_state_df['state_prev7_case100k'])/county_state_df['state_prev7_case100k'])



county_state_df['state_roll7death_diff'] = ((county_state_df['state_curr7_death100k']-county_state_df['state_prev7_death100k'])/county_state_df['state_prev7_death100k'])
county_state_df['state_prevalence_per100k'] = ((county_state_df['state_active']/county_state_df['state_pop'])*100000)

print(county_state_df.head())

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  \
0                0.0    0.0           0.0                0.0   
1                0.0    0.0           0.0                2.0   
2              965.0  356.0         201.0              146.0   
3                0.0    0.0           0.0                0.0   
4                0.0    0.0           0.0                3.0   

             ...             state_curr7_death state_prev7_case  \
0            ...                    115.142857          635.125   
1            ...                     63.714286         4594.750   
2    

# COVID TRACKING PROJ DATA

In [196]:
#########################################
#Bring in COVID TRACKING data
url_all = "https://covidtracking.com/api/v1/states/daily.csv"
testing_all = pd.read_csv(url_all)
testing = testing_all[['date','state','positiveIncrease','negativeIncrease', 'totalTestResultsIncrease']]
testing['state_code'] = testing['state']

print(testing.head())
print(testing.dtypes)

       date state  positiveIncrease  negativeIncrease  \
0  20200630    AK                36              3440   
1  20200630    AL               870             12489   
2  20200630    AR               520              8147   
3  20200630    AS                 0                 0   
4  20200630    AZ              4682             16251   

   totalTestResultsIncrease state_code  
0                      3476         AK  
1                     13359         AL  
2                      8667         AR  
3                         0         AS  
4                     20933         AZ  
date                         int64
state                       object
positiveIncrease             int64
negativeIncrease             int64
totalTestResultsIncrease     int64
state_code                  object
dtype: object


C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [197]:
curr_testing = testing[(testing['date']>=curr_test_end) & (testing['date']<=curr_test_start)]
curr_testing['curr_pos_%'] = (curr_testing['positiveIncrease']/curr_testing['totalTestResultsIncrease'])
print(curr_testing.head(10))

prev_testing = testing[(testing['date']>=prev_test_end) & (testing['date']<=prev_test_start)]
prev_testing['prev_pos_%'] = (prev_testing['positiveIncrease']/prev_testing['totalTestResultsIncrease'])
print(prev_testing.head())

       date state  positiveIncrease  negativeIncrease  \
0  20200630    AK                36              3440   
1  20200630    AL               870             12489   
2  20200630    AR               520              8147   
3  20200630    AS                 0                 0   
4  20200630    AZ              4682             16251   
5  20200630    CA              6367             99080   
6  20200630    CO               204              4088   
7  20200630    CT               152             21266   
8  20200630    DC                35              1900   
9  20200630    DE                98               852   

   totalTestResultsIncrease state_code  curr_pos_%  
0                      3476         AK    0.010357  
1                     13359         AL    0.065125  
2                      8667         AR    0.059998  
3                         0         AS         NaN  
4                     20933         AZ    0.223666  
5                    105447         CA    0.060381  
6

C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [198]:

#Group by with mean
curr_test_grouped = curr_testing.groupby(['state_code'],as_index=False).mean()
prev_test_grouped = prev_testing.groupby(['state_code'], as_index=False).mean()

print(curr_test_grouped.head())
print(prev_test_grouped.head())

  state_code        date  positiveIncrease  negativeIncrease  \
0         AK  20200627.0         23.142857       2725.142857   
1         AL  20200627.0        992.571429       6523.714286   
2         AR  20200627.0        585.571429       6087.428571   
3         AS  20200627.0          0.000000         74.571429   
4         AZ  20200627.0       3005.142857      10114.571429   

   totalTestResultsIncrease  curr_pos_%  
0               2748.285714    0.013895  
1               7516.285714    0.150114  
2               6673.000000    0.090720  
3                 74.571429    0.000000  
4              13119.714286    0.251154  
  state_code        date  positiveIncrease  negativeIncrease  \
0         AK  20200620.0         14.571429       2375.714286   
1         AL  20200620.0        597.857143       6347.000000   
2         AR  20200620.0        498.142857       7919.857143   
3         AS  20200620.0          0.000000          0.000000   
4         AZ  20200620.0       2726.000000 

In [199]:
#Merge to County Set

curr_merge = pd.merge(county_state_df,
                 curr_test_grouped[['state_code','curr_pos_%']],
                 on='state_code', 
                how='left')
print(curr_merge.head())

mam_v2 = pd.merge(curr_merge,
                 prev_test_grouped[['state_code','prev_pos_%']],
                 on='state_code', 
                how='left')


mam_v2['testing_diff'] = (mam_v2['curr_pos_%'] - mam_v2['prev_pos_%'])
print(mam_v2.head())
print(mam_v2.columns)

    fips         County       State state_code Level 6 NBCU Flag  \
0  36061  New York City    New York         NY     NaN         Y   
1  06037    Los Angeles  California         CA     NaN         Y   
2  17031           Cook    Illinois         IL     NaN         Y   
3  48195       Hansford       Texas         TX     NaN       NaN   
4  04013       Maricopa     Arizona         AZ     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services     ...      \
0                0.0    0.0           0.0                0.0     ...       
1                0.0    0.0           0.0                2.0     ...       
2              965.0  356.0         201.0              146.0     ...       
3                0.0    0.0           0.0                0.0     ...       
4                0.0    0.0           0.0                3.0     ...       

   state_prev7_case state_prev7_death state_curr7_case100k  \
0           635.125            39.750             3.046908   
1         

In [200]:
####################################################



county_filtered = mam_v2[(mam_v2['active_cases_100k']>=0)]


#print(county_filtered.active_cases_100k.describe())
#print(county_filtered.roll7case_diff.describe())
#print(county_filtered.roll7death_diff.describe())
#print(county_filtered.state_roll7case_diff.describe())
print(county_filtered.state_prevalence_per100k.describe())


count    2868.000000
mean      136.389203
std        83.517865
min        12.246166
25%        73.532093
50%       105.176100
75%       195.397585
max       543.484890
Name: state_prevalence_per100k, dtype: float64


# SET INDICATORS

In [201]:
#Updated 7/1

def active(n):
    if 0<=n<=49.48: score = 0
    elif 49.49<=n<=114.05: score = 1
    elif 114.06<=n<=100000: score = 2
    else: score = 0
    return(score)



def roll(n):
    if -1<=n<=-0.10: score = 0
    elif -0.10<=n<=0.10: score = 1
    elif 0.101111<=n<=100: score = 2
    else: score = 0
    return(score)

def roll_hrvd(n):
    if -10000<=n<0.10: score = 0
    elif 0.10<=n<=10000: score = 0.5  
    else: score = 0
    return(score)

def roll_hrvd_state(n):
    if -10000<=n<0.10: score = 0
    elif 0.10<=n<=10000: score = 1 
    else: score = 0
    return(score)

def roll_indicator(n):
    if -10000<=n<0.10: score = "Decreasing or Steady"
    elif 0.10<=n<=10000: score = "Increasing"  
    else: score = "Decreasing or Steady"
    return(score)

def fatal(n):
    if n>=0.05: score = 4
    else: score = 0
    return(score)

def test(n):
    if -2<=n<=-.01: score = 0
    elif -.011111<=n<=.01: score = 1
    elif 0.01111 <=n<= 100: score =2
    else: score = 0
    return(score)

def test7(n):
    if -2<=n< 0.03: score = 0
    elif 0.03 <=n<.1: score =1
    elif 0.1 <=n<0.15: score =2
    elif 0.15 <=n<= 1000: score =3
    else: score = 0
    return(score)

def curr7(n):
    if -2<=n<=4.9999: score = 0
    elif 5 <=n<=9.9999: score =1
    elif 10 <=n<=99999: score =2
    else: score = 0
    return(score)

def curr7_hrvd(n):
    if -222<=n<= 1.11111: score = 0
    elif 1 <n<10: score =1
    elif 10 <=n<25: score =2
    elif 25 <=n<=99999: score =3
    else: score = 0
    return(score)

def testch(n):
    if 0<=n<=0.1: score = 'Decreasing'
    elif 1 <=n<= 1.9: score = 'Steady'
    else: score = 'Increasing'
    return(score)

county_filtered['County_Active_100k_Score'] = county_filtered['active_cases_100k'].apply(active)
county_filtered['State_Active_100k_Score'] = county_filtered['state_prevalence_per100k'].apply(active)
county_filtered['County_Roll_Score'] = county_filtered['roll7case_diff'].apply(roll_hrvd)
county_filtered['County Case Trend'] = county_filtered['roll7case_diff'].apply(roll_indicator)

county_filtered['State_Roll_Score'] = county_filtered['state_roll7case_diff'].apply(roll_hrvd_state)
county_filtered['County_Fatality_Roll_Score'] = county_filtered['roll7death_diff'].apply(fatal)
county_filtered['State_Fatality_Roll_Score'] = county_filtered['state_roll7death_diff'].apply(fatal)
county_filtered['State_Testing_Score'] = county_filtered['testing_diff'].apply(test)
county_filtered['State_7Day_Pos_Test_Avg'] = county_filtered['curr_pos_%'].apply(test7)
county_filtered['State_7Day_New_Case_Rolling_Avg'] = county_filtered['state_curr7_case100k'].apply(curr7_hrvd)
county_filtered['County_7Day_New_Case_Rolling_Avg'] = county_filtered['cur_7rollavg_cases'].apply(curr7_hrvd)


C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [202]:
county_filtered['State_Composite_Score'] =  ( county_filtered['State_Roll_Score']
                                             +county_filtered['State_7Day_New_Case_Rolling_Avg']
                                             +county_filtered['State_7Day_Pos_Test_Avg'])

county_filtered['County_Composite_Score'] = (county_filtered['County_Roll_Score'] + county_filtered['County_7Day_New_Case_Rolling_Avg'])



C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [203]:
def county(n):
    if 0<=n<=.99: score = "Minimal"
    elif 0.1<=n<=1: score = "Low"
    elif 1<n<=2: score = "Moderate"
    elif 2<n<=3: score = "Severe"
    elif 4<=n<= 100: score = "Extreme"
    else: score = 0
    return(score)

def county_hrvd(n):
    if 0<=n<0.5: score = "Minimal"
    elif 0.5<=n<1: score = "Low"
    elif 1<=n<2: score = "Moderate"
    elif 2<=n<3: score = "Severe"
    elif 3<=n<= 100: score = "Extreme"
    else: score = 0
    return(score)


def state(n):
    if 0<=n<1: score = "Minimal"
    elif 1<=n<2: score = "Low"
    elif 2<=n<4: score = "Moderate"
    elif 4<=n<6: score = "Severe"
    elif 6<=n<= 100: score = "Extreme"
    else: score = 0
    return(score)

def blend(n):
    if 0<=n<=.99: score = "Minimal"
    elif 1<=n<=2: score = "Low"
    elif 3<=n<=4: score = "Moderate"
    elif 5<=n<=6: score = "Severe"
    elif 7 <=n<= 100: score = "Extreme"
    else: score = 0
    return(score)

county_filtered['County_Level'] = county_filtered['County_Composite_Score'].apply(county_hrvd)
county_filtered['State_Level'] = county_filtered['State_Composite_Score'].apply(state)
county_filtered['County_State_Blend'] = ((county_filtered['County_Composite_Score']+county_filtered['State_Composite_Score']))
county_filtered['County_State_Level'] = county_filtered['County_State_Blend'].apply(blend)
print(county_filtered.head())



C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


    fips       County       State state_code Level 6 NBCU Flag  \
1  06037  Los Angeles  California         CA     NaN         Y   
2  17031         Cook    Illinois         IL     NaN         Y   
3  48195     Hansford       Texas         TX     NaN       NaN   
4  04013     Maricopa     Arizona         AZ     NaN         Y   
5  06073    San Diego  California         CA     NaN         Y   

   Tech Ops EE Count    CAE  Cable Stores  Business Services  \
1                0.0    0.0           0.0                2.0   
2              965.0  356.0         201.0              146.0   
3                0.0    0.0           0.0                0.0   
4                0.0    0.0           0.0                3.0   
5                0.0    0.0           0.0                0.0   

          ...          State_Testing_Score State_7Day_Pos_Test_Avg  \
1         ...                            1                       1   
2         ...                            1                       0   
3       

C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ccolam127\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [214]:
county_fields = county_filtered[['fips','County','State','County_Level', 'State_Level', 'County_Composite_Score',
                                 'cur_7rollavg_cases','County Case Trend',                    
                                 'Total CCC','CAE','Cable Stores','Business Services', 'Tech Ops EE Count']]

print(county_fields.head())
county_fields.to_csv(file,index=False) 

    fips       County       State County_Level State_Level  \
1  06037  Los Angeles  California       Severe      Severe   
2  17031         Cook    Illinois     Moderate    Moderate   
3  48195     Hansford       Texas       Severe     Extreme   
4  04013     Maricopa     Arizona      Extreme     Extreme   
5  06073    San Diego  California       Severe      Severe   

   County_Composite_Score  cur_7rollavg_cases County Case Trend  Total CCC  \
1                     2.5           21.725040        Increasing       98.0   
2                     1.5            7.633497        Increasing     3640.0   
3                     2.5           10.583971        Increasing        0.0   
4                     3.5           47.222906        Increasing        9.0   
5                     2.5           12.217400        Increasing       23.0   

     CAE  Cable Stores  Business Services  Tech Ops EE Count  
1    0.0           0.0                2.0                0.0  
2  356.0         201.0          

# Create State Only File 

In [205]:
#Merge state data with state base file 

state_active= pd.merge(state_pop_df,
                 state_prevalence_grouped[['State','state_active',
                                       ]],
                 on='State', 
                how='left')


state_current_w_pop = pd.merge(state_active,
                 state_current_grouped[['State','state_curr7_case',
                                       'state_curr7_death']],
                 on='State', 
                how='left')


state_df = pd.merge(state_current_w_pop,
                 state_previous_grouped[['State','state_prev7_case','state_prev7_death']],
                 on='State', 
                how='left')
print(state_df.head(10))

                  State state_code  Comcast Employees  state_pop   CAE  \
0               Alabama         AL                407    4908621    65   
1                Alaska         AK                  0    3038999     0   
2               Arizona         AZ                744    7378494   237   
3              Arkansas         AR                117    3038999     1   
4            California         CA               5022   39937489     0   
5              Colorado         CO               8452    5845526  1206   
6           Connecticut         CT               1270    3563077   306   
7              Delaware         DE               1132     982895   499   
8  District of Columbia         DC                360     720687     0   
9               Florida         FL               7451   21992985  2123   

   Cable Stores  Business Services  Tech Ops  comcast_state  state_active  \
0            40                  9       178              1       11133.0   
1             0                

In [206]:
#Create State Level metrics

state_df['state_curr7_case100k'] = ((state_df['state_curr7_case']/state_df['state_pop'])*100000)
state_df['state_curr7_death100k'] = ((state_df['state_curr7_death']/state_df['state_pop'])*100000)
state_df['state_prev7_case100k'] = ((state_df['state_prev7_case']/state_df['state_pop'])*100000)
state_df['state_prev7_death100k'] = ((state_df['state_prev7_death']/state_df['state_pop'])*100000)


state_df['state_roll7case_diff'] = ((state_df['state_curr7_case100k']-state_df['state_prev7_case100k'])/state_df['state_prev7_case100k'])
state_df['state_roll7death_diff'] = ((state_df['state_curr7_death100k']-state_df['state_prev7_death100k'])/state_df['state_prev7_death100k'])
state_df['state_prevalence_per100k'] = ((state_df['state_active']/state_df['state_pop'])*100000)

print(state_df.head())

        State state_code  Comcast Employees  state_pop  CAE  Cable Stores  \
0     Alabama         AL                407    4908621   65            40   
1      Alaska         AK                  0    3038999    0             0   
2     Arizona         AZ                744    7378494  237            17   
3    Arkansas         AR                117    3038999    1            13   
4  California         CA               5022   39937489    0           491   

   Business Services  Tech Ops  comcast_state  state_active  \
0                  9       178              1       11133.0   
1                  0         0              0         381.0   
2                  5        87              1       40101.0   
3                  7        64              1        7586.0   
4                219      1997              1       73022.0   

             ...             state_curr7_death  state_prev7_case  \
0            ...                     12.285714           644.000   
1            ...      

In [207]:
# Merge in Covid Tracking Data


curr_merge = pd.merge(state_df,
                 curr_test_grouped[['state_code','curr_pos_%']],
                 on='state_code', 
                how='left')

state_mam_v2 = pd.merge(curr_merge,
                 prev_test_grouped[['state_code','prev_pos_%']],
                 on='state_code', 
                how='left')


state_mam_v2['testing_diff'] = (state_mam_v2['curr_pos_%'] - state_mam_v2['prev_pos_%'])
print(state_mam_v2.head())

        State state_code  Comcast Employees  state_pop  CAE  Cable Stores  \
0     Alabama         AL                407    4908621   65            40   
1      Alaska         AK                  0    3038999    0             0   
2     Arizona         AZ                744    7378494  237            17   
3    Arkansas         AR                117    3038999    1            13   
4  California         CA               5022   39937489    0           491   

   Business Services  Tech Ops  comcast_state  state_active      ...       \
0                  9       178              1       11133.0      ...        
1                  0         0              0         381.0      ...        
2                  5        87              1       40101.0      ...        
3                  7        64              1        7586.0      ...        
4                219      1997              1       73022.0      ...        

   state_curr7_case100k  state_curr7_death100k  state_prev7_case100k  \
0 

In [208]:
# State Level Scores


state_mam_v2['State_Active_100k_Score'] = state_mam_v2['state_prevalence_per100k'].apply(active)
state_mam_v2['State_Roll_Score'] = state_mam_v2['state_roll7case_diff'].apply(roll_hrvd_state)
state_mam_v2['State_Fatality_Roll_Score'] = state_mam_v2['state_roll7death_diff'].apply(fatal)
state_mam_v2['State_Testing_Score'] = state_mam_v2['testing_diff'].apply(test)
state_mam_v2['State_7Day_Pos_Test_Avg'] = state_mam_v2['curr_pos_%'].apply(test7)
state_mam_v2['State_7Day_New_Case_Rolling_Avg'] = state_mam_v2['state_curr7_case100k'].apply(curr7_hrvd)



In [209]:
#Create final dataset

state_mam_v2['State Composite Score'] =  (  state_mam_v2['State_Roll_Score']
                                             + state_mam_v2['State_7Day_New_Case_Rolling_Avg']
                                             +state_mam_v2['State_7Day_Pos_Test_Avg'])

state_mam_v2['State Classification'] = state_mam_v2['State Composite Score'].apply(state)
state_mam_v2['Test_Change_Classification'] = state_mam_v2['State_Testing_Score'].apply(testch)



final_state_df = state_mam_v2.query('comcast_state ==1')
print(final_state_df.head())
print(final_state_df.columns)

        State state_code  Comcast Employees  state_pop   CAE  Cable Stores  \
0     Alabama         AL                407    4908621    65            40   
2     Arizona         AZ                744    7378494   237            17   
3    Arkansas         AR                117    3038999     1            13   
4  California         CA               5022   39937489     0           491   
5    Colorado         CO               8452    5845526  1206           246   

   Business Services  Tech Ops  comcast_state  state_active  \
0                  9       178              1       11133.0   
2                  5        87              1       40101.0   
3                  7        64              1        7586.0   
4                219      1997              1       73022.0   
5                709       763              1        3271.0   

              ...              testing_diff  State_Active_100k_Score  \
0             ...                  0.060373                        2   
2       

In [213]:
#Export State File to CSV

final_state_df2 = final_state_df[['State', 'State Classification', 'State Composite Score','state_curr7_case100k','curr_pos_%',
                                  'state_roll7case_diff', 'State_Roll_Score',
                                 'State_7Day_Pos_Test_Avg','State_7Day_New_Case_Rolling_Avg',
                                 'Comcast Employees','CAE','Cable Stores','Business Services', 'Tech Ops']]

                                  
print(final_state_df2.head())
final_state_df2.to_csv(state_file, index=False) 

        State State Classification  State Composite Score  State_Roll_Score  \
0     Alabama              Extreme                      6                 1   
2     Arizona              Extreme                      7                 1   
3    Arkansas               Severe                      4                 1   
4  California               Severe                      4                 1   
5    Colorado             Moderate                      3                 1   

   state_roll7case_diff  State_7Day_Pos_Test_Avg  curr_pos_%  \
0              0.541260                        3    0.150114   
2              0.147940                        3    0.251154   
3              0.119639                        1    0.090720   
4              0.278291                        1    0.059760   
5              0.215621                        1    0.051402   

   State_7Day_New_Case_Rolling_Avg  state_curr7_case100k  Comcast Employees  \
0                                2             20.220983     

In [211]:
#################FOR SCORE CREATION#########


#state_mam_v3 = state_mam_v2[['State', 'State Classification', 'State Composite Score', 'state_prevalence_per100k',
#                                 'state_roll7case_diff','Test_Change_Classification','State_7Day_Pos_Test_Avg','State_7Day_New_Case_Rolling_Avg',
#                                 'Comcast Employees','CAE','Cable Stores','Business Services', 'Tech Ops','testing_diff','state_curr7_case100k','curr_pos_%','state_prev7_case100k']]



In [212]:
#state_mam_v3.to_csv('for_scoring_629.csv') 